#### 1. Obtención base de datos - Web Scraping

In [1]:
import pandas as pd
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')
from selenium.webdriver.common.by import By
from IPython.display import Image # Para mostrar imágenes en python
from bs4 import BeautifulSoup as bs
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

pd.set_option('display.max_columns', None)

In [2]:
from selenium import webdriver
PATH = 'driver/chromedriver.exe'
driver = webdriver.Chrome()

Cache folder (C:\Users\jacom\.cache\selenium) cannot be created: No se puede crear un archivo que ya existe. (os error 183)
Cache folder (C:\Users\jacom\.cache\selenium) cannot be created: No se puede crear un archivo que ya existe. (os error 183)
Cache folder (C:\Users\jacom\.cache\selenium) cannot be created: No se puede crear un archivo que ya existe. (os error 183)


In [3]:
# Definimos URL de búsqueda
url = 'https://www.theca.org.uk/orcas/reports'

In [4]:
driver.get(url)

In [5]:
driver.find_element(By.XPATH, '/html/body/div[5]/div/div/div[2]/button[2]').click() # Clickamos en "No cookies, thanks"

In [6]:
# Clickamos en "SHOW LIST"
time.sleep(2)
driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[1]/strong').click()

In [7]:
# Clickamos en primera pestaña con datos del caso más reciente
time.sleep(1)
driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[4]/div[1]/p').click()


In [8]:
# Defino una lista vacía donde voy a ir appendeando la fecha, lat. y long. y las respuestas de cada pregunta del cuestionario que responden los 
# skippers de cada embarcación que ha experienciado una interacción con una orca:
questionnaire_list = []

In [51]:
# Appendeo fecha
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[4]').text)
time.sleep(0.5)
# Appendeo lat. y long. juntos
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[5]').text)
time.sleep(0.5)
# Appendeo pregunta 1: GTOA Protocol
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[7]').text)
time.sleep(0.5)
# Appendeo pregunta 2: Boat Type
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[9]').text)
time.sleep(0.5)
# Appendeo pregunta 3: Boat length (metres)
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[11]').text)
time.sleep(0.5)
# Appendeo pregunta 4: Towing an inflatable?
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[13]').text)
time.sleep(0.5)
# Appendeo pregunta 5: Trailing fishing lure?
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[15]').text)
time.sleep(0.5)
# Appendeo pregunta 6: Did orcas make physical contact with boat
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[17]').text)
time.sleep(0.5)
# Appendeo pregunta 7: Number of adult orcas seen?
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[19]').text)
time.sleep(0.5)
# Appendeo pregunta 8: Rudder type
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[21]').text)
time.sleep(0.5)
# Appendeo pregunta 9: Motoring or sailing?
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[23]').text)
time.sleep(0.5)
# Appendeo pregunta 10: Speed (knots)
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[25]').text)
time.sleep(0.5)
# Appendeo pregunta 11: Sea state
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[27]').text)
time.sleep(0.5)
# Appendeo pregunta 12: Wind Speed (Beaufort)
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[29]').text)
time.sleep(0.5)
# Appendeo pregunta 13: Darkness or daylight
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[31]').text)
time.sleep(0.5)
# Appendeo pregunta 14: Cloud cover
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[33]').text)
time.sleep(0.5)
# Appendeo pregunta 15: Distance off land (nautical miles)
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[35]').text)
time.sleep(0.5)
# Appendeo pregunta 16: Depth (metres)
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[37]').text)
time.sleep(0.5)
# Appendeo pregunta 17:Depth gauge ON?
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[39]').text)
time.sleep(0.5)
# Appendeo pregunta 18: Autopilot ON?
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[41]').text)
time.sleep(0.5)
# Appendeo pregunta 19: Hull topside colour
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[43]').text)
time.sleep(0.5)
# Appendeo pregunta 20: Antifoul colour
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[45]').text)
time.sleep(0.5)
# Appendeo pregunta 21: Was the boat damaged and in need of a repair?
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[47]').text)
time.sleep(0.5)
# Appendeo pregunta 22: Tow required
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[49]').text)
time.sleep(0.5)
# Appendeo pregunta 23: Crew response
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[51]').text)
time.sleep(0.5)
# Appendeo pregunta 24: Orca behaviour
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[53]').text)
time.sleep(0.5)
# Appendeo pregunta 25: Stage of day
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[55]').text)
time.sleep(0.5)
# Appendeo pregunta 26: Tide
questionnaire_list.append(driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[2]/p[59]').text)

In [52]:
questionnaire_list

['2023-11-01 22:15:00',
 '32 47.4980 N, 9 54.3980 W',
 'No, did not follow protocol, interaction lasted less than 10 minutes',
 'Sail',
 '10 - 12.5m',
 'Not towing',
 'No',
 'No',
 '1',
 'Spade',
 'Sailing',
 '5 - 7',
 'Moderate',
 '5 - 6 (17 - 27 knots)',
 'Night',
 '0 - 25%',
 'Over 10',
 '200m+',
 'On',
 'On',
 'White/light',
 'Blue',
 'No',
 'No',
 "Orca interaction at 10:15pm on 01/11, 40 miles off the coast of Safi, on passage from Tangier to the Canary Islands. We were relaxing in the cockpit with a podcast playing quite loudly on a portable speaker. It was pitch black as the moon had not yet risen. A lone adult orca (to our knowledge) approached our yacht (Beneteau Oceanis 411) and surfaced about half a metre from our stern. We were only alerted to it's presence when it's blowhole spouted, roughly 0.5 metres off the left side of our stern, right next to our navigation light. It was so close to us that the water from its blowhole hit our chart plotter. We stood up to inspect and

* Vale guay, tengo una lista que compondría el valor de cada una de las columnas para la primera fila - la última interacción orca-barco.

* Ahora el objetivo de esto sería meter las 26 respuestas a una lista dentro de un bucle , y que cada paso por bucle rellene una lista. El bucle se repetirá tantas veces como interacciones de orca haya.

In [57]:
# Voy a comprobar a ver si puedo buclear por las tabs de cada interacción:

# Cerramos caso actual
time.sleep(1)
driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[1]/span[1]').click()


In [60]:
# Dentro de cada bucle se tendrá que abrir la tab del caso -> leer info y rellenar una lista -> cerrar tab
for i in range(1, 4, 1):
    time.sleep(1)
    driver.find_element(By.XPATH, f'/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[4]/div[{i}]/p').click()
    print('Acabo de abrir tab')
    time.sleep(1)
    print('Simulo que estoy leyendo info y cierro tab')
    time.sleep(1)
    driver.find_element(By.XPATH, '/html/body/div[3]/div/section/div/section/div/div[5]/div/div/div/div[2]/div/div/div[2]/div[2]/div[3]/div[3]/div[1]/span[1]').click()



Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab
Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab
Acabo de abrir tab
Simulo que estoy leyendo info y cierro tab
